## Explicit Fact Queries

In [24]:
import os
import time
import openai
from openai import AzureOpenAI
from typing import List, Dict, Any

# Azure Cognitive Search Imports
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    HnswParameters,
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField
)

# ----------------------------
# 1. CONFIGURATION
# ----------------------------

# Replace with your actual Azure Cognitive Search endpoint and admin key
AZURE_SEARCH_ENDPOINT = "https://.search.windows.net"
AZURE_SEARCH_KEY =  ""


client = AzureOpenAI(
  api_key = "",  
  api_version = "2024-02-01",
  azure_endpoint = "https://.openai.azure.com/" 
)

# Index name to store documents
INDEX_NAME = "product-documents-index"


EMBEDDING_MODEL_NAME = "text-embedding-ada-002"  # e.g. "text-embedding-ada-002" or your custom Azure deployment
GPT_DEPLOYMENT_NAME = "gpt-4o"     # e.g. "gpt-35-turbo" or "gpt-4" or your custom Azure deployment

# Create clients
credential = AzureKeyCredential(AZURE_SEARCH_KEY)

# Index client (to create, update, and delete indexes)
search_index_client = SearchIndexClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=credential)

# Search client (to add documents, query, etc.)
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=credential)


# ----------------------------
# 2. DATA PREPARATION & CHUNKING
# ----------------------------

def chunk_text_by_paragraphs(text: str, max_chars: int = 500) -> List[str]:
    """
    Chunk text into segments. Preserves paragraphs if under max_chars; otherwise splits them.
    """
    paragraphs = text.split("\n\n")
    chunks = []
    current_chunk = ""

    for para in paragraphs:
        if len(current_chunk) + len(para) <= max_chars:
            if not current_chunk:
                current_chunk = para
            else:
                current_chunk += "\n\n" + para
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Example data
documents_data = [
    {
        "id": "doc1",
        "title": "Azure AI Search Overview",
        "url": "https://docs.microsoft.com/en-us/azure/search",
        "text": """Azure AI Search is a search-as-a-service solution with built-in AI capabilities for cloud apps... 
                   It provides both full-text search using BM25 and vector search using embeddings..."""
    },
    {
        "id": "doc2",
        "title": "Azure OpenAI Introduction",
        "url": "https://docs.microsoft.com/en-us/azure/openai",
        "text": """Azure OpenAI offers access to advanced language models like GPT, enabling developers to build intelligent apps...
                   Embeddings allow for semantic understanding, powering advanced search solutions."""
    }
]

# Create chunks
all_chunks = []
for d in documents_data:
    text_chunks = chunk_text_by_paragraphs(d["text"], max_chars=500)
    for i, chunk in enumerate(text_chunks):
        all_chunks.append({
            "vector_id": f"{d['id']}-{i}",
            "id": d["id"],
            "url": d["url"],
            "title": d["title"],
            "text": chunk
        })


# ----------------------------
# 3. CREATING / UPDATING THE INDEX
# ----------------------------

def create_or_update_index(index_name: str):
    """
    Create or update an index with:
      - Fields for metadata (id, url, title, text)
      - Two vector fields (title_vector, content_vector) using HNSW
      - Semantic search configuration
    """

    fields = [
        # Non-key fields
        SimpleField(name="id", type=SearchFieldDataType.String),
        SimpleField(name="url", type=SearchFieldDataType.String),
        SearchableField(name="title", type=SearchFieldDataType.String),
        SearchableField(name="text", type=SearchFieldDataType.String),
        # KEY field: unique vector_id for each chunk
        SimpleField(name="vector_id", type=SearchFieldDataType.String, key=True),

        # Vector fields
        SearchField(
            name="title_vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=1536,
            vector_search_profile_name="my-vector-config"
        ),
        SearchField(
            name="content_vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=1536,
            vector_search_profile_name="my-vector-config"
        )
    ]

    # Vector search config
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="my-hnsw",
                kind=VectorSearchAlgorithmKind.HNSW,
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE,
                ),
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="my-vector-config",
                algorithm_configuration_name="my-hnsw",
            )
        ],
    )

    # Semantic search config
    semantic_search = SemanticSearch(
        configurations=[
            SemanticConfiguration(
                name="my-semantic-config",
                prioritized_fields=SemanticPrioritizedFields(
                    title_field=SemanticField(field_name="title"),
                    keywords_fields=[SemanticField(field_name="url")],
                    content_fields=[SemanticField(field_name="text")]
                ),
            )
        ]
    )

    # Create the index
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search
    )

    # Create or update in Azure Cognitive Search
    try:
        result = search_index_client.create_or_update_index(index)
        print(f"Index {result.name} created/updated successfully.")
    except Exception as e:
        print("Error creating/updating index:", e)

# Execute the creation/update
create_or_update_index(INDEX_NAME)



# ----------------------------
# 4. GENERATE EMBEDDINGS & UPLOAD DOCUMENTS
# ----------------------------

def generate_embeddings(texts: List[str]) -> List[List[float]]:
    response = client.embeddings.create(
        model=EMBEDDING_MODEL_NAME,
        input=texts
    )
    return response.data[0].embedding

def generate_embeddings_for_docs(texts: list[str]) -> list[list[float]]:
    # 'client' is your AzureOpenAI or OpenAI client
    response = client.embeddings.create(
        model=EMBEDDING_MODEL_NAME,  # e.g., "text-embedding-ada-002"
        input=texts
    )
    # Each item.embedding is already a one-dimensional list of floats
    # So this returns a list of lists: [[0.12, -0.08, ...], [0.09, 0.01, ...], ...]
    return [item.embedding for item in response.data]

def upload_documents(chunks: List[Dict[str, Any]]):
    """
    For each chunk of data, generate embeddings for title and text, then upload to Azure Search index.
    """
    # Prepare arrays for batch embedding calls
    titles = [c["title"] for c in chunks]
    contents = [c["text"] for c in chunks]

    # Generate embeddings in bulk
    title_embeddings = generate_embeddings_for_docs(titles)
    content_embeddings = generate_embeddings_for_docs(contents)

    # Build documents with vector fields
    documents_to_upload = []
    for i, chunk in enumerate(chunks):
        doc = {
            "vector_id": chunk["vector_id"],
            "id": chunk["id"],
            "url": chunk["url"],
            "title": chunk["title"],
            "text": chunk["text"],
            "title_vector": title_embeddings[i],
            "content_vector": content_embeddings[i]
        }
        documents_to_upload.append(doc)

    # Upload to Azure Cognitive Search
    try:
        result = search_client.upload_documents(documents=documents_to_upload)
        print("Upload result:", result)
    except Exception as e:
        print("Error uploading documents:", e)

upload_documents(all_chunks)

# Allow some time for indexing
time.sleep(5)


# ----------------------------
# 5. RETRIEVAL
# ----------------------------

# Only the relevant modified code, using VectorizedQuery for pure vector search:

from azure.search.documents.models import VectorizedQuery

def search_query(query: str, top_k: int = 3):
    # Generate the embedding for the query (assuming you have generate_embeddings already defined)
    query_embedding = generate_embeddings(query)

    # Construct the vector query
    vector_query = VectorizedQuery(
        vector=query_embedding,
        k_nearest_neighbors=top_k,
        fields="content_vector"
    )

    # Perform the search
    results = search_client.search(
        search_text=None,
        vector_queries=[vector_query],
        select=["title", "text", "url"]
    )

    # Process and return results
    output = []
    for result in results:
        output.append({
            "title": result["title"],
            "text": result["text"],
            "url": result["url"],
            "score": result["@search.score"]
        })

    return output

# Example usage:
query =  "What does Azure AI Search offer?"
docs = search_query(query, top_k=3)
for doc in docs:
    print(f"Title: {doc['title']}")
    print(f"Score: {doc['score']}")
    print(f"URL: {doc['url']}\n")


# 6. ANSWER GENERATION (BASIC RAG)
# ----------------------------

def generate_answer_with_context(query: str, retrieved_docs: List[Dict[str, Any]]) -> str:
    # Combine text from retrieved docs
    context_str = "\n\n".join(
        [f"Doc {i+1} (Title: {doc['title']}):\n{doc['text']}" for i, doc in enumerate(retrieved_docs)]
    )

    system_message = (
        "You are a helpful AI assistant. Please answer the user's query based on the provided context. "
        "If the context does not contain the answer, say you are unsure."
    )
    user_message = f"Question: {query}\n\nContext:\n{context_str}\n\nAnswer:"

    response = client.chat.completions.create(
        model=GPT_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0.2,
        max_tokens=300
    )

    return response.choices[0].message.content.strip()

answer = generate_answer_with_context(user_query, docs)
print("\nGenerated Answer:")
print(answer)



# ----------------------------
# 7. IMPROVEMENTS & EXAMPLES
# ----------------------------

# --- Example: Recursive Retrieval ---
def recursive_retrieval(query: str, max_iterations: int = 2) -> str:
    current_query = query
    for i in range(max_iterations):
        print(f"--- Iteration {i+1} ---")
        docs = search_query(current_query, top_k=2)
        ans = generate_answer_with_context(current_query, docs)
        print(f"Answer at iteration {i+1}:\n{ans}\n")

        # Simple approach: if we detect "I'm not sure" or "unsure", refine the query
        if "unsure" in ans.lower():
            current_query = f"{query} (More details needed)"
        else:
            return ans
    return ans

# Example usage
final_answer = recursive_retrieval("Explain how to use Azure AI Search for vector search.")
print("Final Answer:", final_answer)


Index product-documents-index created/updated successfully.
Upload result: [<azure.search.documents._generated.models._models_py3.IndexingResult object at 0x0000019B59C9E450>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x0000019B59C9DF10>]
Title: Azure AI Search Overview
Score: 0.9084782
URL: https://docs.microsoft.com/en-us/azure/search

Title: Azure OpenAI Introduction
Score: 0.8701985
URL: https://docs.microsoft.com/en-us/azure/openai


Generated Answer:
Azure AI Search offers a search-as-a-service solution with built-in AI capabilities for cloud apps. It provides both full-text search using BM25 and vector search using embeddings.
--- Iteration 1 ---
Answer at iteration 1:
To use Azure AI Search for vector search, you can leverage the built-in AI capabilities that support embeddings. Here’s a general approach to using Azure AI Search for vector search:

1. **Generate Embeddings**: Use a model, such as those provided by Azure OpenAI, to generate e